In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# verileri hazır hale getirme
training_dir="D:\yapay zeka projeler/computer_vision/fire_detection/dataset/Training"
validation_dir="D:\yapay zeka projeler/computer_vision/fire_detection/dataset/Validation"



In [3]:
input_shape=(224,224,3)

In [4]:
# Veri arttırma Tekniklerii uygulama
training_data_generator=ImageDataGenerator(rescale=1./255,horizontal_flip=True,vertical_flip=True,rotation_range=45,height_shift_range=0.2,width_shift_range=0.2,fill_mode="nearest")

validation_data_generator=ImageDataGenerator(rescale=1./255)

# ilgili directory ye gidip dataları çoğaltma
train_generator=training_data_generator.flow_from_directory(training_dir,target_size=(224,224),class_mode="categorical",batch_size=64)
validation_generator=validation_data_generator.flow_from_directory(validation_dir,target_size=(224,224),class_mode="categorical",batch_size=16)


Found 980 images belonging to 2 classes.
Found 239 images belonging to 2 classes.


In [5]:
from keras import layers
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
def fireNet(input_shape):
# Model Oluşturduğumuz fonksiyon
    model = Sequential(
        [
            layers.Conv2D(96,(11,11),strides=(4,4),activation="relu",input_shape=input_shape),
            layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)) ,

            layers.Conv2D(256,(5,5),activation="relu"),
            layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)) ,

            layers.Conv2D(512,(5,5),activation="relu"),
            layers.MaxPooling2D(pool_size=(3,3),strides=(2,2)) ,

            layers.Flatten(),
            layers.Dropout(0.3),

            layers.Dense(2048,activation="relu"),
            layers.Dropout(0.3),

            layers.Dense(1024,activation="relu"),
            layers.Dropout(0.3),

            layers.Dense(2,activation="softmax")

        ]
    )

    model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=1e-4), metrics=["acc"])

    return model

In [6]:
model=fireNet(input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 512)         3277312   
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 512)        0

In [8]:
# Eğitim - Fitting

history =model.fit(train_generator, steps_per_epoch=15 , epochs=10,validation_data=validation_generator,validation_steps=15)

Epoch 1/10
15/15 [==============================] - 79s 5s/step - loss: 0.5597 - acc: 0.6867 - val_loss: 0.4355 - val_acc: 0.7197
Epoch 2/10
15/15 [==============================] - 59s 4s/step - loss: 0.3728 - acc: 0.8210 - val_loss: 0.3351 - val_acc: 0.9038
Epoch 3/10
15/15 [==============================] - 55s 4s/step - loss: 0.2878 - acc: 0.8788 - val_loss: 0.2430 - val_acc: 0.9163
Epoch 4/10
15/15 [==============================] - 55s 4s/step - loss: 0.2729 - acc: 0.8919 - val_loss: 0.2291 - val_acc: 0.9247
Epoch 5/10
15/15 [==============================] - 53s 4s/step - loss: 0.2366 - acc: 0.9203 - val_loss: 0.2195 - val_acc: 0.9372
Epoch 6/10
15/15 [==============================] - 58s 4s/step - loss: 0.2311 - acc: 0.9061 - val_loss: 0.2078 - val_acc: 0.9331
Epoch 7/10
15/15 [==============================] - 54s 4s/step - loss: 0.2244 - acc: 0.9192 - val_loss: 0.1925 - val_acc: 0.9414
Epoch 8/10
15/15 [==============================] - 55s 4s/step - loss: 0.2173 - acc: 0.92

In [8]:
# Görselleştirme

import matplotlib.pyplot as plt

acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs=range(0,50)

plt.plot(epochs,acc,"g",label="Training Accuracy")
plt.plot(epochs,val_acc,"r",label="Validation Accuracy")
plt.title("Training | Validation Accuracy")
plt.legend(loc=0)
plt.figure()
plt.show()

plt.plot(epochs,loss,"blue",label="Training Loss")
plt.plot(epochs,val_loss,"brown",label="Validation Loss")
plt.title("Training | Validation Loss")
plt.legend(loc=0)
plt.figure()
plt.show()

NameError: name 'history' is not defined

In [54]:
#model.save("D:\yapay zeka projeler/computer_vision/fire_detection/models/fire_detection_model.h5")

In [2]:
# Prediction Images

import cv2
import numpy as np
from keras.models import  load_model

In [3]:
model=load_model("models/fire_detection_model.h5")
path ="test/test.jpg"
video_path ="test/test3.mp4"

In [4]:

test_img=cv2.imread(path)
#Resmi arraya dönüştürüp yeniden boyutlandırma işlemine tabi tuttuk
img=np.asarray(test_img)
img=cv2.resize(img,(224,224))
# normalizasyon işlemi - yani 0-1 arasına sıkıştırdık
img=img/255
print(img.shape)

# görüntüyü 4 boyuta çevirmem lazım
img=img.reshape(1,224,224,3)


(224, 224, 3)


In [5]:
# Prediction(Image)

predictions=model.predict(img)
pred=np.argmax(predictions[0])

probability = predictions[0][pred]
probability_="% {:.2f}".format(probability*100)

if pred==1:
    label="Yangin Var"
else:
    label="Normal"

font=cv2.FONT_HERSHEY_SIMPLEX
color=(0,255,0)
cv2.putText(test_img,label,(35,60),font,1,color,2)
cv2.putText(test_img,probability_,(20,100),font,1,color,2)

cv2.imshow("Prediction : ",test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# Predict (Video)
cap=cv2.VideoCapture(video_path)

while True:
    ret,frame=cap.read()
    img = np.asarray(frame)
    img = cv2.resize(img, (224, 224))
# normalizasyon işlemi - yani 0-1 arasına sıkıştırdık
    img = img / 255
    #print(img.shape)

# görüntüyü 4 boyuta çevirmem lazım
    img = img.reshape(1, 224, 224, 3)

# Prediction(Image)

    predictions = model.predict(img)
    pred = np.argmax(predictions[0])

    probability = predictions[0][pred]
    probability_ = "% {:.2f}".format(probability * 100)

    if pred == 1:
        label = "Yangin Var"
    else:
        label = "Normal"

    font = cv2.FONT_HERSHEY_SIMPLEX
    color = (0, 255, 0)
    cv2.putText(frame, label, (35, 60), font, 1, color, 2)
    cv2.putText(frame, probability_, (20, 100), font, 1, color, 2)

    cv2.imshow("Prediction : ", frame)

    if cv2.waitKey(10) & 0xFF==ord("q"):
        break
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: Expected cv::UMat for argument 'src'